# let's install necessary libraies

In [ ]:
!pip install langchain pypdf sentence-transformers faiss-cpu

# let's extract all the contents from the pdf:
- The link to the pdf (https://pgcag.files.wordpress.com/2010/01/48lawsofpower.pdf).

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("pgcag_file.pdf")
full_texts_from_pdf = loader.load_and_split()


In [4]:
full_texts_from_pdf[0]

Document(page_content='The 48 Laws Of Power \n \n \n \n \nBy  \nRobert Greene \n \n \n \n \nPenguin Books 2000 \nHardcover Edition ISBN 0-670-88146-5 \nPaperback Edition ISBN 0 14 02.8019 7 \n452 pages \n \n \nWISDOM IN A NUTSHELL', metadata={'source': 'E:\\llm_langchai_learning\\Betterzila _Changumangu_Solutions_LLP_task\\pgcag_file.pdf', 'page': 0})

In [24]:
len(full_texts_from_pdf)

17

# let's convert the full text into small chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
spliter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n","."],
    chunk_size = 600,
    chunk_overlap = 80,
    length_function =len,
)
chunks = spliter.split_documents(full_texts_from_pdf)

In [6]:
chunks[0]

Document(page_content='The 48 Laws Of Power \n \n \n \n \nBy  \nRobert Greene \n \n \n \n \nPenguin Books 2000 \nHardcover Edition ISBN 0-670-88146-5 \nPaperback Edition ISBN 0 14 02.8019 7 \n452 pages \n \n \nWISDOM IN A NUTSHELL', metadata={'source': 'E:\\llm_langchai_learning\\Betterzila _Changumangu_Solutions_LLP_task\\pgcag_file.pdf', 'page': 0})

# let's initialize the embedding method
- here we are using HuggingFaceInferenceAPIEmbeddings.
- model = all-MiniLM-l6-v2

In [7]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embed = HuggingFaceInferenceAPIEmbeddings(
    api_key='paste your hugging face api key',
    model_name="sentence-transformers/all-MiniLM-l6-v2")

# let's store the embeddings in vectore data base
- here we are using faiss

In [8]:
from langchain.vectorstores import FAISS
vectorindex = FAISS.from_documents(chunks, embed)

# let's save the data base so we can directly use it locally and build an streamlit application

In [3]:
import pickle
file_path = "vector_index_file.pkl"

# now let's load the data from vector base and build the retriever.

In [4]:
with open(file_path, "rb") as index_file:
  loaded_vector_index = pickle.load(index_file)

In [5]:
retriever = loaded_vector_index.as_retriever(score_threshold = 0.7)

In [6]:
rdocs = retriever.get_relevant_documents("Can you give me an example from history where the enemy was crushed totally from the book")
rdocs


[Document(page_content='plot their revenge. \n• Banish enemies or plot for the best time to render them harmless. \n• Leave your enemies no options. \n• Sometimes enemies will destroy themselves.  \n• Thoughts of reconciliation will open you up to attack. \n \n \n \n16. USE ABSENCE TO INCRE ASE RESPECT AND HONOR. \n• Observance of the Law', metadata={'source': 'temp_uploaded_file.pdf', 'page': 7}),
 Document(page_content='best friend Michael III murdered. \n \n \n \n• Observance of the Law \nEmperor Sung of China in 959 A.D. was able to turn all his enemies into loyal friends. \nHe persuaded his generals to retire to a life of nobility and give up their dreams of', metadata={'source': 'temp_uploaded_file.pdf', 'page': 1}),
 Document(page_content='he devised a clever last resort plan. Lia ng donned a Taoist robe and played the lute \nupon the wall of the city he was defending. His soldiers opened the gates and hid. \nWhen Sima Yi’s troops advanced upon t he unguarded city, Yi recognized

# let's initialize google palm model

In [7]:
from langchain.llms import GooglePalm

llm = GooglePalm(google_api_key='paste your api key',temperature=0.4, max_tokens=200)
llm.invoke("write a 4 line poem on Rahul gandhi?")

e:\llm_langchai_learning\llm_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Rahul Gandhi,\nA man of hope and change,\nFighting for the people,\nLeading India to a better future.'

# now let's write a template and combine all together with chain

In [27]:
from langchain.prompts import PromptTemplate

prompt_template = """Please generate an answer by extracting relevant information from the provided context.
Aim to include as much text as possible from the "response" section in the source document context without making significant changes.
If the information is not available then respond  "I don't know." don't make it your own answers

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [9]:
import langchain
langchain.debug = True

# Now let's ask one question from the pdf

In [28]:
query = "who is narendra modi?"
response = chain(query)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "who is narendra modi?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "who is narendra modi?",
  "context": "was extensive. He was so invo lved in so many areas of th e political structure that to \nremove him would lead to chaos. The intens ive form of power provides more freedom \nthan the extensive. \n \nMachiavelli said it is better to be feared than loved. Fear can be controlled; love, never. \n \nWisdom in a nutshell: \n• Be the only one who can do what you do. Make the fate of those who hire you so \nentwined with yours they cannot  possibly get rid of you. \n• If you are ambitious, it is wiser to seek out weak masters with whom you can \ncreate a relationship of dependency.\n\nbooks and lectures on the langua

In [21]:
response

{'query': 'who is narendra modi?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='was extensive. He was so invo lved in so many areas of th e political structure that to \nremove him would lead to chaos. The intens ive form of power provides more freedom \nthan the extensive. \n \nMachiavelli said it is better to be feared than loved. Fear can be controlled; love, never. \n \nWisdom in a nutshell: \n• Be the only one who can do what you do. Make the fate of those who hire you so \nentwined with yours they cannot  possibly get rid of you. \n• If you are ambitious, it is wiser to seek out weak masters with whom you can \ncreate a relationship of dependency.', metadata={'source': 'temp_uploaded_file.pdf', 'page': 6}),
  Document(page_content='books and lectures on the language, culture and history of the island of Formosa (now \nTaiwan).  After his death, it was revealed t hat he was in fact a Frenchman with a rich \nimagination who played on the British need for

In [34]:
query = "what is earth?"
response = chain(query)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "what is earth?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "what is earth?",
  "context": "• Fit the spirit of the times. \n• Be a source of pleasure. \n \n25. RE-CREATE YOURSELF. \nObservance of the Law\n\nmessages.\n \n• People are always impressed by the superf icial appearance of things, the grand, and \nthe spectacular, what is larger than life. \n \n38. THINK AS YOU LIKE BUT BEHAVE LIKE OTHERS. \nWisdom in a nutshell: \n• \nWise and clever people learn early on that they can display conventional behavior \nand mouth conventional ideas without having to believe in them.\n \n• \nPut on the mask appropriate to the group you are joining.\n \n \n39. STIR UP WATERS TO CATCH FISH. \nWisdom in a nutshell: \n• \nAng

In [33]:
query = "What's the point of making myself less accessible?"
response = chain(query)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What's the point of making myself less accessible?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What's the point of making myself less accessible?",
  "context": "isolation. \n \nWisdom in a nutshell: \n• A fortress may be impregnable, but ever yone knows you are there and it may easily \nturn into a prison.  \n• Power depends on social interaction and circulation.  \n• Isolation is deadly for the creative arts. Shakespeare was always producing plays for \nthe masses. He was in constant touch with reality and what people wanted.  \n• Mobility and social contact protects you from plotters.  \n \n19. KNOW WHO YOU’RE DEALING WITH – DO NOT OFFEND THE \nWRONG PERSON. \n• Transgression of the Law\n\nshow, the more supe